In [58]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
df = pd.DataFrame({"col1":[-0.73,-2.36, 2.17, np.nan, -0.82, 1.83,  1.66, np.nan,  0.78 ,  1.56],
                   "col2":[ 0.21, -0.089 , -1.34,  2.57,  1.81, -1.77, -0.28, -0.73, -0.99, -1.54],
                   "col3":[-0.92, np.nan,  1.92 , -1.42 , -0.38, 1.47,  np.nan, -1.79,  np.nan,  1.22],
                   "cat":["B", "B", "M", "B", "B", "M", np.nan, "B", "M", "M"],
                   "target":[0, 0, 1, 0, 0, 1, 1, 0, 1, 1]})


In [59]:
#sns.pairplot(df[["col1", "col2", "col3", "target"]], hue='target')
X = df.drop("target", axis=1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.3)

In [60]:
ct = ColumnTransformer([("mean_col1", SimpleImputer(strategy="mean"), ["col1"]),
                        ("median_col2", SimpleImputer(strategy="median") , ["col2"]),
                        ("most_freq_cat", SimpleImputer(strategy="most_frequent") , ["cat"])
                        ])

In [62]:
pl = Pipeline(steps=[("ct", ct),
                     ("est", KNeighborsClassifier())])


In [63]:
pl

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('mean_col1', SimpleImputer(),
                                                  ['col1']),
                                                 ('median_col2',
                                                  SimpleImputer(strategy='median'),
                                                  ['col2']),
                                                 ('most_freq_cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['cat'])])),
                ('est', KNeighborsClassifier())])

In [64]:
pl.fit(X_train, y_train)
pl.predict(X_test)

ValueError: could not convert string to float: 'M'

Me faltaría convertir la categórica a numérica....pero perdí el nombre de la columna, puedo usar el número de la columna

In [65]:
temp = pl[:-1].fit_transform(X_train)
temp

array([[2.17, -1.34, 'M'],
       [1.56, -1.54, 'M'],
       [1.66, -0.28, 'B'],
       [1.088, -0.73, 'B'],
       [-0.73, 0.21, 'B'],
       [1.088, 2.57, 'B'],
       [0.78, -0.99, 'M']], dtype=object)

In [66]:
oh = OneHotEncoder(sparse_output=False, dtype=int)
oh.fit_transform(X_train[["cat"]])

array([[0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]])

Es importante ver que por paralelización todas las operaciones se hacen de manera concurrente, si quiero primero imputar la categorica y luego transformarla en numérica necesito otro ct

In [67]:
ct = ColumnTransformer([("mean_col1", SimpleImputer(strategy="mean"), ["col1"]),
                        ("median_col2", SimpleImputer(strategy="median") , ["col3"]),
                        ("most_freq_cat", SimpleImputer(strategy="most_frequent") , ["cat"]),
                        #("OneHot", OneHotEncoder(sparse_output=False, dtype=int, drop='first') , ["cat"]) <= Esto no funciona
                        ], remainder='passthrough')
#Aca ya perdí el nombre de la columna :(
#ct2 = ColumnTransformer([("OneHot", OneHotEncoder(sparse_output=False, dtype=int, drop='first') , ["cat"])])
ct2 = ColumnTransformer([("OneHot", OneHotEncoder(sparse_output=False, dtype=int, drop='first') , [2])],#<=Tengo que poner el indice de la columna
                         remainder='passthrough')

In [68]:
temp = ct.fit_transform(X_train)
temp

array([[2.17, 1.92, 'M', -1.34],
       [1.56, 1.22, 'M', -1.54],
       [1.66, -0.92, 'B', -0.28],
       [1.088, -1.79, 'B', -0.73],
       [-0.73, -0.92, 'B', 0.21],
       [1.088, -1.42, 'B', 2.57],
       [0.78, -0.92, 'M', -0.99]], dtype=object)

In [69]:
pl = Pipeline(steps=[("ct", ct),
                     ("ct2", ct2),
                     ("est", KNeighborsClassifier())])
temp = pl[:-1].fit_transform(X_train)
temp

array([[1, 2.17, 1.92, -1.34],
       [1, 1.56, 1.22, -1.54],
       [0, 1.66, -0.92, -0.28],
       [0, 1.088, -1.79, -0.73],
       [0, -0.73, -0.92, 0.21],
       [0, 1.088, -1.42, 2.57],
       [1, 0.78, -0.92, -0.99]], dtype=object)

In [70]:
column_types = [type(temp[0, i]) for i in range(temp.shape[1])]
print(column_types)  # [<class 'int'>, <class 'str'>, <class 'float'>]

[<class 'int'>, <class 'float'>, <class 'float'>, <class 'float'>]


In [71]:
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

